# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to start by delegating the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. They have the skills and expertise to handle this task effectively.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL",
"context": "The Data Analyst will need to use statistical modeling and machine learning techniques to identify trends and predict market movements for AAPL. They should provide insights and alerts about significant market opportunities or threats."
}


> Entering new CrewAgentExecutor chain...
I need to start by gathering current market data for AAPL to analyze trends and predict move

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication Se

Final Answer: The Data Analyst will continuously monitor and analyze market data for AAPL using statistical modeling and machine learning techniques to identify trends and predict market movements. They will provide insights and alerts about significant market opportunities or threats for AAPL.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: The Data Analyst will continuously monitor and analyze market data for AAPL using statistical modeling and machine learning techniques to identify trends and predict market movements. They will provide insights and alerts about significant market opportunities or threats for AAPL.
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Develop and refine trading strategies based on the insights from the Data Analyst and user-defined risk tolerance (Medium). Consider trading preferences (Day Trading).


> Entering new CrewAgentExecutor chain...
I need to delegate tasks to the Data Analyst and Trading Strategy Developer to develop trad

Thought: I have gathered the latest market data for AAPL from Yahoo Finance. Now, I need to analyze this data using statistical modeling and machine learning techniques to provide insights for potential trading strategies.

Final Answer: After analyzing the market data for AAPL from Yahoo Finance, I have observed that the stock price has been relatively stable with a slight decrease. The key events, financial highlights, and statistical measures indicate that AAPL is a profitable company with a strong market cap. Based on this analysis, potential trading strategies could involve buying AAPL stock for long-term investment or selling options to capitalize on the stable stock price. However, further in-depth analysis using statistical modeling and machine learning techniques is recommended for a more accurate prediction of market movements.

> Finished chain.
 

After analyzing the market data for AAPL from Yahoo Finance, I have observed that the stock price has been relatively stable wit

Final Answer: 
After analyzing the market data for AAPL, the stock price for Apple Inc. (AAPL) is currently at 221.25 USD. The company designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It offers various products and services, including iPhone, Mac, iPad, wearables, home, and accessories, as well as platforms like the App Store and Apple Music. AAPL has a market cap of 3.364T and a trailing P/E ratio of 33.68. The stock has shown positive returns over different periods, with a YTD return of 15.35% and a 1-year return of 25.08%. It is recommended to consider these insights along with user-defined risk tolerance for day trading strategies involving AAPL.

> Finished chain.
 

After analyzing the market data for AAPL, the stock price for Apple Inc. (AAPL) is currently at 221.25 USD. The company designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It offers various 

Final Answer: 

After analyzing the market data for AAPL and considering the insights provided by the Data Analyst and user-defined risk tolerance for day trading, potential trading strategies for AAPL could involve buying AAPL stock for long-term investment or selling options to capitalize on the stable stock price. It is recommended to continuously monitor market trends, news related to AAPL, and historical price data to adjust the strategies accordingly and maximize profit while minimizing risk.

To further develop detailed execution plans for trading AAPL, I have gathered information from various sources:

1. MarketWatch: AAPL | Complete Apple Inc. stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview.
Link: https://www.marketwatch.com/investing/stock/aapl

2. CNBC: Get Apple Inc (AAPL:NASDAQ) real-time stock quotes, news, price, and financial information from CNBC.
Link: https://www.cnbc.com/quotes/AAPL

3. Yahoo Finance: Get the lat

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

I apologize for the limitations in accessing specific website content for the risk analysis of trading strategies for AAPL. To conduct a comprehensive risk analysis and suggest mitigation strategies, I recommend exploring alternative sources such as financial news websites like MarketWatch, CNBC, Yahoo Finance, Nasdaq, CNN, WSJ, TipRanks.com, and Nasdaq Insights. Additionally, analyzing market trends, historical price data, and incorporating insights from the Data Analyst will be crucial in developing effective risk mitigation strategies for the proposed trading activities. Continuously monitoring these sources and staying informed about AAPL's stock performance will enable us to maximize profit while minimizing risks associated with the trading strategies.